In [1]:
import string

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
# import scipy.stats as ss
# import nltk
# nltk.download("stopwords")
# nltk.download('wordnet')
# nltk.download('vader_lexicon')

from transformers import pipeline

# from nltk.stem import PorterStemmer
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
# from nltk import FreqDist
# from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
drop_col_emo=['occupation', 'usage', 'recruitment', 'comfort']

# Students at LAB42

In [3]:
emo_sl42 = pd.read_csv('data/lab42_cleaned_data.csv', index_col=0).drop(columns = drop_col_emo, axis=1).dropna(subset=['emotion'])

In [4]:
emo_sl42

,floor,ground-floor,1-floor,2-floor,3-floor,emotion
1,NaN,NaN,NaN,NaN,NaN,I always feel a bit zen when I am here. It’s q...
2,NaN,NaN,NaN,NaN,NaN,"Cold, a bit down"
3,NaN,NaN,NaN,NaN,NaN,Calm
4,6th Floor,NaN,NaN,NaN,NaN,"Hungry, a bit cold"
5,Ground floor,Library learning room,NaN,NaN,NaN,Stressed about my exams but I am happy in this...
...,...,...,...,...,...,...
148,1st Floor,NaN,Green group study tables (between a row of pla...,NaN,NaN,"I’m a little cold, but otherwise calm and comf..."
149,1st Floor,NaN,Yellow/white chairs & tables (besides the wood...,NaN,NaN,Relaxed and focused
150,Ground floor,Green chairs by the entrance,NaN,NaN,NaN,Pretty relaxed
152,1st Floor,NaN,Green group study tables (between a row of pla...,NaN,NaN,Comfortable


# LOAD MODEL

In [5]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=MODEL, tokenizer=MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def sentiment_analysis(sentence):
    return sentiment_task(sentence, top_k=3)


def assign_scores(idx):
    emo_sl42.at[idx, 'pos'] = next(item for item in emo_sl42['senti'][idx] if item['label'] == 'positive')['score']
    emo_sl42.at[idx, 'neg'] = next(item for item in emo_sl42['senti'][idx] if item['label'] == 'negative')['score']
    emo_sl42.at[idx, 'neutral'] = next(item for item in emo_sl42['senti'][idx] if item['label'] == 'neutral')['score']


In [ ]:
# 1. Calculate sentiment analysis on each emotion statement:
emo_sl42['senti'] = emo_sl42['emotion'].apply(sentiment_analysis)

In [10]:

for idx in emo_sl42.head().index:
    assign_scores(idx)

In [ ]:
emo_sl42

In [ ]:
sentiment_cols = ['positive', 'negative', 'neutral']
emo_sl42[[sentiment_cols]] = 0

In [ ]:
def plot_sentiment_analysis(df):
    sns.countplot(y=df["label"], 
                  data=df,
                  palette=['#b2d8d8',"#008080", '#db3d13'],
                  order=["positive", "negative", "neutral"])
    plt.ylabel("Sentiment for emo Text")
    plt.xlabel("Count")
    # plt.xlim(0,45)
#     plt.savefig('plots/employees-' + label + '-sentiment-count.jpg',
#                 dpi=300, bbox_inches='tight')
    
#     plt.clf()
    
#     sns.boxplot(x='floor', y='compound', data=df)
#     plt.xlabel("Floor")
#     plt.ylabel("Sentiment Value \n (1 = Most Positive; 0 = Neutral; -1 = Most Negative)")
#     plt.tick_params(bottom=False)
#     plt.axhline(0, ls='--', c = 'grey')
#     plt.savefig('plots/employees-' + label + '-sentiment-distribution.jpg',
#                 dpi=300, bbox_inches='tight')

In [ ]:
plot_sentiment_analysis(df_emo)